In [1]:
print('start l2 activity')
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import growthepieapi_utils as gtp
import l2beat_utils as ltwo
sys.path.pop()

import numpy as np
import pandas as pd

start l2 activity


In [2]:
# # # Usage
gtp_api = gtp.get_growthepie_api_data()
# # print(merged_df.sample(5))
gtp_meta_api = gtp.get_growthepie_api_meta()

In [3]:
l2beat_df = ltwo.get_all_l2beat_data()
l2beat_meta = ltwo.get_l2beat_metadata()
l2beat_meta['chain'] = l2beat_meta['slug']

/Users/michaelsilberling/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/l2beat_utils.py:181: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)
/Users/michaelsilberling/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/l2beat_utils.py:181: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)


In [4]:
# print(l2beat_df[l2beat_df['chain']=='degen'].sample(5))
# print(l2beat_meta[l2beat_meta['layer']=='layer2'].sample(5))
# print(l2beat_meta[l2beat_meta['layer']=='layer3'].sample(5))

In [5]:
# display(l2beat_meta.sample(5))

In [6]:
combined_l2b_df = l2beat_df.merge(l2beat_meta[['chain','name','layer','chainId','provider','category','is_upcoming']], on='chain',how='left')
# combined_l2b_df.tail(5)

In [7]:
combined_gtp_df = gtp_api.merge(gtp_meta_api[['origin_key','chain_name']], on='origin_key',how='left')
# combined_gtp_df.sample(5)

In [8]:
# Check Columns
# Assuming combined_gtp_df is your DataFrame
column_names = combined_gtp_df.columns

for col in column_names:
    if col.endswith('_usd'):
        # Construct the new column name by replacing '_usd' with '_eth'
        new_col_name = col.replace('_usd', '_eth')
        
        # Check if the new column name exists in the DataFrame
        if new_col_name not in combined_gtp_df.columns:
            # If it doesn't exist, create the column and fill it with nan values
            combined_gtp_df[new_col_name] = np.nan

In [9]:
# print(combined_gtp_df.dtypes)
# print(l2beat_df.dtypes)
# combined_gtp_df.sample(5)
combined_l2b_df.sample(5)

,timestamp,valueUsd,cbvUsd,ebvUsd,nmvUsd,valueEth,cbvEth,ebvEth,nmvEth,chain,transactions,ethereumTransactions,name,layer,chainId,provider,category,is_upcoming
46288,2023-06-05,254449.70,254449.70,0.0,0.0,134.542028,134.542028,0.0,0.0,harmony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73990,2022-10-21,416940.07,416940.07,0.0,0.0,325.215475,325.215475,0.0,0.0,wormholeV1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21415,2023-04-02,17319529.21,17319529.21,0.0,0.0,9500.562532,9500.562532,0.0,0.0,nova,145793.0,981226.0,Arbitrum Nova,L2,None,Arbitrum Orbit,None,False
27630,2023-01-11,16879116.91,16879116.91,0.0,0.0,12637.220451,12637.220451,0.0,0.0,deversifi,89.0,1083178.0,NaN,NaN,NaN,NaN,NaN,NaN
14159,2024-01-23,2334.71,2334.71,0.0,0.0,1.009842,1.009842,0.0,0.0,karak,16.0,1083406.0,Karak,L2,None,OP Stack,None,False


In [10]:
# Add Metadata
opstack_metadata = opstack_metadata = pd.read_csv('../op_chains_tracking/outputs/chain_metadata.csv')
opstack_metadata['chain'] = opstack_metadata['l2beat_slug']
meta_cols = ['chain', 'is_op_chain','mainnet_chain_id','op_based_version', 'alignment','chain_name']

l2b_enriched_df = combined_l2b_df.merge(opstack_metadata[meta_cols], on='chain', how = 'left')

l2b_enriched_df['alignment'] = l2b_enriched_df['alignment'].fillna('Other EVMs')
l2b_enriched_df['is_op_chain'] = l2b_enriched_df['is_op_chain'].fillna(False)

In [17]:
#  Define aggregation functions for each column
aggregations = {
    'valueUsd': ['min', 'last', 'mean'],
    'transactions': ['sum', 'mean'],
    'cbvUsd': ['min', 'last', 'mean'],
    'ebvUsd': ['min', 'last', 'mean'],
    'nmvUsd': ['min', 'last', 'mean'],
}

# Group by month, chain, layer, and other specified columns and apply aggregations
l2b_monthly_df = l2b_enriched_df.groupby([pd.Grouper(key='timestamp', freq='MS'), 'chain', 'layer', 'is_op_chain', 'mainnet_chain_id', 'op_based_version', 'alignment', 'chain_name'], dropna=False).agg(aggregations).reset_index()

# Flatten the hierarchical column index and concatenate aggregation function names with column names
l2b_monthly_df.columns = [f'{col}_{func}' if func != '' else col for col, func in l2b_monthly_df.columns]

# Rename the 'date' column
l2b_monthly_df.rename(columns={'timestamp': 'month'}, inplace=True)
l2b_monthly_df.sample(5)

,month,chain,layer,is_op_chain,mainnet_chain_id,op_based_version,alignment,chain_name,valueUsd_min,valueUsd_last,...,transactions_mean,cbvUsd_min,cbvUsd_last,cbvUsd_mean,ebvUsd_min,ebvUsd_last,ebvUsd_mean,nmvUsd_min,nmvUsd_last,nmvUsd_mean
541,2021-12-31,opticsV1,NaN,False,NaN,NaN,Other EVMs,NaN,7.898636e+06,7.898636e+06,...,NaN,7.898636e+06,7.898636e+06,1.398942e+07,0.0,0.0,0.0,0.0,0.0,0.0
1393,2023-02-28,polygon-pos,NaN,False,NaN,NaN,Other EVMs,NaN,2.574227e+09,2.628691e+09,...,NaN,2.574227e+09,2.628691e+09,2.646825e+09,0.0,0.0,0.0,0.0,0.0,0.0
936,2022-07-31,pNetwork,NaN,False,NaN,NaN,Other EVMs,NaN,6.762707e+06,8.607181e+06,...,NaN,6.762707e+06,8.607181e+06,8.191498e+06,0.0,0.0,0.0,0.0,0.0,0.0
488,2021-11-30,multichain,NaN,False,NaN,NaN,Other EVMs,NaN,2.939247e+09,3.045265e+09,...,NaN,2.939247e+09,3.045265e+09,3.092716e+09,0.0,0.0,0.0,0.0,0.0,0.0
27,2020-01-31,metis,L2,False,1088.0,legacy ovm,OP Stack Fork,metis andromeda,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# export
folder = 'outputs/'
combined_gtp_df.to_csv(folder + 'growthepie_l2_activity.csv', index = False)
gtp_meta_api.to_csv(folder + 'growthepie_l2_metadata.csv', index = False)
l2b_enriched_df.to_csv(folder + 'l2beat_l2_activity.csv', index = False)
l2beat_meta.to_csv(folder + 'l2beat_l2_metadata.csv', index = False)
l2b_monthly_df.to_csv(folder + 'l2beat_l2_activity_monthly.csv', index = False)
# Post to Dune API
d.write_dune_api_from_pandas(combined_gtp_df, 'growthepie_l2_activity',\
                             'L2 Usage Activity from GrowThePie')
d.write_dune_api_from_pandas(gtp_meta_api, 'growthepie_l2_metadata',\
                             'L2 Metadata from GrowThePie')
d.write_dune_api_from_pandas(l2b_enriched_df, 'l2beat_l2_activity',\
                             'L2 Usage Activity from L2Beat')
d.write_dune_api_from_pandas(l2b_monthly_df, 'l2beat_l2_activity_monthly',\
                             'Monthly L2 Usage Activity from L2Beat')
d.write_dune_api_from_pandas(l2beat_meta, 'l2beat_l2_metadata',\
                             'L2 Metadata from L2Beat')


table at: dune.oplabspbc.dataset_growthepie_l2_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"growthepie_l2_activity"}'
table at: dune.oplabspbc.dataset_growthepie_l2_metadata
Response status code: 200
Response content: b'{"success":true,"table_name":"growthepie_l2_metadata"}'
table at: dune.oplabspbc.dataset_l2beat_l2_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_activity"}'
table at: dune.oplabspbc.dataset_l2beat_l2_activity_monthly
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_activity_monthly"}'
table at: dune.oplabspbc.dataset_l2beat_l2_metadata
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_metadata"}'
